In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
    
printables = ['conversation_id','creation_date_time','a_b_test','response','text']

def get_convs(filename):
    header = ['user_id','conversation_id','creation_date_time','a_b_test','chosenModule','response','request_type','text']
    data = pd.read_csv(filename, sep='\t', names=header, header=None)
    return data

In [2]:
data = get_convs('statetable.txt')

In [3]:
data.conversation_id.nunique()

4539

In [4]:
grouped_df = data.groupby('conversation_id').a_b_test
conversation_id = []
version = []
for key, item in grouped_df:
    conversation_id.append(key)
    version.append(grouped_df.get_group(key).to_list()[0])

In [5]:
version_data = pd.DataFrame.from_dict({'conversation_id': conversation_id, 'version': version})

In [6]:
ratings = pd.read_csv('ratings.csv', sep=',', names=['conversation_id','start','rating'], header=0)

In [7]:
ratings_df_final = pd.merge(version_data, ratings, on="conversation_id",how="left")
ratings_df_final = ratings_df_final[['conversation_id','version','rating']]

In [8]:
version_count = ratings_df_final.version.value_counts().to_frame()
version_count.index.name = 'version' 
version_count.columns = ['count']
version_count

,count
version,
None,2320
IrisDec4,1336
EmoraDec4,882


In [9]:
to_drop = ratings_df_final[ ratings_df_final['rating'].isnull() ].index
ratings_df_final_filtered = ratings_df_final.drop(to_drop)
summary_counts = ratings_df_final_filtered.version.value_counts().to_frame()
summary_counts.index.name = 'version' 
summary_counts.columns = ['count']

summary_score = ratings_df_final.groupby('version').agg({'rating':['mean']}).astype(float)
summary_score = pd.merge(summary_score, summary_counts, on="version")

/anaconda2/envs/nlp/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
summary_score

,"(rating, mean)",count
version,,
EmoraDec4,3.241848,184
IrisDec4,3.281677,322
None,2.939515,701
